**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/
STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size = 16
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
df_train_toxic = pd.read_csv(DATA_DIR+'jigsaw-toxic-comment-train.csv')
df_train_toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df_train_bias = pd.read_csv(DATA_DIR + 'jigsaw-unintended-bias-train.csv')
df_train_bias.head()

,id,comment_text,toxic,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
# len(df_train_toxic[df_train_toxic['toxic']==1]),len(df_train_toxic[df_train_toxic['toxic']==0])
# len(df_train_bias[df_train_bias['toxic']<0.1])/7,len(df_train_bias[df_train_bias['toxic']>0.80])

In [7]:
df_train_bias_pos = df_train_bias[df_train_bias['toxic']>0.8].reset_index(drop=True)
df_train_bias_neg = shuffle(df_train_bias[df_train_bias['toxic']<0.1].reset_index(drop=True)).reset_index(drop=True).iloc[::7]
df_train_bias =  df_train_bias_pos.append(df_train_bias_neg).reset_index(drop=True)

In [8]:
df_valid = shuffle(pd.read_csv(DATA_DIR + 'validation.csv'))
df_valid.head()

,id,comment_text,lang,toxic
6707,6707,"Merhabe Ömer, İzberbaş maddesinin sağ üst köşe...",tr,0
2956,2956,Sei pronto al conto alla rovescia ... ^^ ...,it,0
760,760,El Ronaldo este tiene la misma cara que el que...,es,0
1155,1155,"¿Que usted es un intelectual 100%, dice? ¡Qué ...",es,1
1919,1919,:Resim:Akor-nomen.gif lisans problemi :Resi...,tr,0


In [9]:
df_valid.groupby(['lang', 'toxic']).count()

id  comment_text
lang toxic                    
es   0      2078          2078
     1       422           422
it   0      2012          2012
     1       488           488
tr   0      2680          2680
     1       320           320

In [10]:
df_test = pd.read_csv(DATA_DIR + 'test.csv')
df_test.head()

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [11]:
df_test.groupby(['lang']).count()

,id,content
lang,,
es,8438,8438
fr,10920,10920
it,8494,8494
pt,11012,11012
ru,10948,10948
tr,14000,14000


In [12]:
len(df_test)

63812

In [13]:
df_sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
df_sub.head()

,id,toxic
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [14]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [15]:
target_columns = 'toxic'

### Define dataset

In [16]:
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = DistilBertTokenizer.from_pretrained('bert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
        #distilbert-base-multilingual-cased

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)

    def trim_input(self, text, max_sequence_length=MAX_LEN):
        t = self.tokenizer.tokenize(text)
        t_len = len(t)

        if t_len + 2 > max_sequence_length:

            t_new_len = int(max_sequence_length) - 2

            t = t[:t_new_len]

        return t
        
    def get_token_ids(self, row):
        t_tokens = self.trim_input(row.comment_text)

#         tokens = ['[CLS]'] + t_tokens  + ['[SEP]']+ t_tokens[-1::-1]+ ['[SEP]']
        tokens = ['[CLS]'] + t_tokens  + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        if len(token_ids) < MAX_LEN:
            token_ids += [0] * (MAX_LEN - len(token_ids))
            
        ids = torch.tensor(token_ids)
        
        return ids

    def get_label(self, row):
#         label = torch.tensor(row[target_columns].astype(np.long))
        label = np.round(row[target_columns])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}

/home/kb/jig_env/lib/python3.7/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/kb/jig_env/lib/python3.7/site-packages/torchvision/_C.so'>



## Build Model

In [17]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased') 
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.bert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
    
        self.fc = nn.Linear(768, n_classes)

    def forward(self, ids):
        attention_mask = (ids > 0)
        layers = self.bert_model(input_ids=ids, attention_mask=attention_mask)
        
        out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
        logit = self.fc(out)#.unsqueeze(1)
        return logit #, 'for_auc': logit[:, 1]}#[:,1]
    


In [18]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        if self.callback_get_label:
            return self.callback_get_label(dataset, idx)
        else:
            dataset.labels[idx]
#             raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [19]:
dataframes = {
    'bias': df_train_bias,
    'toxic': df_train_toxic,
    'valid': df_valid,
#     'valid2': df_test
}
def get_loaders(train_pt='bias', valid_pt='valid', test_pt=None, to_balance=True, shuffle_before=True):
    if SERVER:
        workers=1
    else:
        workers = 6
    
    if isinstance(train_pt, list):
        df_train = dataframes[train_pt[0]][['comment_text', 'toxic']]
        for pt in train_pt[1:]:
            df_train = df_train.append(dataframes[pt][['comment_text', 'toxic']]).reset_index(drop=True)
    else:
        df_train = dataframes[train_pt][['comment_text', 'toxic']]
    
    if shuffle_before:
        df_train = shuffle(df_train)
    
    
    train_dataset = QuestDataset(df_train.iloc[::STRIDE], train_mode=True)
    valid_dataset = QuestDataset(dataframes[valid_pt], train_mode=False)
    if test_pt != None:
        test_dataset = QuestDataset(dataframes[test_pt].iloc[::STRIDE], train_mode=False)
    
    train_loader = DataLoader(
        train_dataset,
        num_workers=workers,
#         sampler=ImbalancedDatasetSampler(train_dataset) if to_balance else None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=workers,
        batch_size=batch_size,
    )    
    if test_pt != None:
        test_loader = DataLoader(
            test_dataset,
            num_workers=workers,
            batch_size=batch_size,
        )
       
    loaders = {}
    loaders['train'] = train_loader
    
    loaders['valid'] = valid_loader
    
    if test_pt != None:
        loaders['valid2'] = test_loader
    
    for i in ['es', 'it', 'tr']:
        df = dataframes[valid_pt]
        df = df[df['lang']==i]

        loaders['valid_'+ i] = DataLoader(
            QuestDataset(df, train_mode=False),
            num_workers=workers,
            batch_size=batch_size,
        )
    
    
    return loaders


In [20]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:

project = "jigsaw_v4_distilbert"
num_epochs = 30

group = datetime.now().strftime("%m_%d_%Y__%H_%M_%S")

    
if SERVER:
    group = f'{group}_server'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'

variants = {
#     'learn_both':{
#         'train': ['bias', 'toxic'],
#         'valid': 'valid',
#         'valid2': 'valid'
#     },
    'learn_toxic':{
        'train': 'toxic',
        'valid': 'valid',
#         'valid2': 'bias',
    },
#     'learn_bias':{
#         'train': 'bias',
#         'valid': 'valid',
#         'valid2': 'toxic',
#     },
    
    
}
gradient_accumulation_steps = 1

lr = 0.0001

group += f'_lr{lr}'

group += f'_nosampler'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

for experiment in variants.keys():
    torch.cuda.empty_cache()
    
    logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

    model = QuestModel(2)

#     model = model.to(device)

    # model runner
    if 'valid2' in variants[experiment].keys():
        test_var= variants[experiment]['valid2']
    else:
        test_var=None

    loaders = get_loaders(variants[experiment]['train'], 
                          variants[experiment]['valid'],
                          test_var, to_balance=True)
            
    
    t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
    warmup_proportion = 0.01
    num_warmup_steps = t_total * warmup_proportion
    
    criterion = torch.nn.BCEWithLogitsLoss()
#     criterion = torch.nn.CrossEntropyLoss()
#     criterion = torch.nn.BCELoss()
    optimizer = AdamW(model.parameters(), lr = lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.25)
    print(f'----------------Experiment: {experiment}')

    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        loaders=loaders,
        logdir=logdir,
        num_epochs=num_epochs,
        verbose=True,
        distributed=False if is_jupyter() else True,
        callbacks=[
            AlchemyLogger(
                    token=token, # your Alchemy token
                    project=project,
                    experiment=experiment,
                    group=group,
                ),
            MyAUCCallback()
#             AUCCallback(input_key = 'targets',
#                                 output_key = 'logits',
#                                 prefix = 'auc',
#                                 class_names = None,
#                                 num_classes = 2,
#                                 activation = 'Sigmoid',)
        ]

    )

----------------Experiment: learn_toxic
1/30 * Epoch (train):  26% 3623/13972 [16:59<48:28,  3.56it/s, loss=0.784]  

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 994, in _validate_c

1/30 * Epoch (train): 100% 13972/13972 [1:05:26<00:00,  3.56it/s, loss=0.801]
1/30 * Epoch (valid): 100% 500/500 [00:44<00:00, 11.17it/s, loss=0.789]
1/30 * Epoch (valid_es): 100% 157/157 [00:14<00:00, 11.09it/s, loss=0.748]
1/30 * Epoch (valid_it): 100% 157/157 [00:14<00:00, 11.08it/s, loss=0.833]
1/30 * Epoch (valid_tr): 100% 188/188 [00:16<00:00, 11.09it/s, loss=0.814]
[2020-04-16 01:46:56,775] 
1/30 * Epoch 1 (_base): lr=2.386e-08 | momentum=0.9000
1/30 * Epoch 1 (train): auc/_mean=0.4367 | auc/class_0=0.4367 | loss=0.7869
1/30 * Epoch 1 (valid): auc/_mean=0.5174 | auc/class_0=0.5174 | es_auc/_mean=0.4805 | es_auc/class_0=0.4805 | es_loss=0.7631 | it_auc/_mean=0.5134 | it_auc/class_0=0.5134 | it_loss=0.7758 | loss=0.7774 | tr_auc/_mean=0.5259 | tr_auc/class_0=0.5259 | tr_loss=0.7909
1/30 * Epoch 1 (valid_es): auc/_mean=0.4805 | auc/class_0=0.4805 | loss=0.7631
1/30 * Epoch 1 (valid_it): auc/_mean=0.5134 | auc/class_0=0.5134 | loss=0.7758
1/30 * Epoch 1 (valid_tr): auc/_mean=0.5259 

/home/kb/jig_env/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:224: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



2/30 * Epoch (train): 100% 13972/13972 [1:05:26<00:00,  3.56it/s, loss=0.083]
2/30 * Epoch (valid): 100% 500/500 [00:44<00:00, 11.17it/s, loss=0.438]
2/30 * Epoch (valid_es): 100% 157/157 [00:14<00:00, 11.07it/s, loss=0.506]
2/30 * Epoch (valid_it): 100% 157/157 [00:14<00:00, 11.08it/s, loss=0.304]
2/30 * Epoch (valid_tr): 100% 188/188 [00:16<00:00, 11.08it/s, loss=0.360]
[2020-04-16 02:53:58,959] 
2/30 * Epoch 2 (_base): lr=4.771e-08 | momentum=0.9000
2/30 * Epoch 2 (train): auc/_mean=0.5556 | auc/class_0=0.5556 | loss=0.4176
2/30 * Epoch 2 (valid): auc/_mean=0.5094 | auc/class_0=0.5094 | es_auc/_mean=0.5769 | es_auc/class_0=0.5769 | es_loss=0.4582 | it_auc/_mean=0.5380 | it_auc/class_0=0.5380 | it_loss=0.4959 | loss=0.4650 | tr_auc/_mean=0.5028 | tr_auc/class_0=0.5028 | tr_loss=0.4440
2/30 * Epoch 2 (valid_es): auc/_mean=0.5769 | auc/class_0=0.5769 | loss=0.4582
2/30 * Epoch 2 (valid_it): auc/_mean=0.5380 | auc/class_0=0.5380 | loss=0.4959
2/30 * Epoch 2 (valid_tr): auc/_mean=0.5028 

10/30 * Epoch (train): 100% 13972/13972 [1:05:24<00:00,  3.56it/s, loss=0.010]
10/30 * Epoch (valid): 100% 500/500 [00:44<00:00, 11.15it/s, loss=0.671]
10/30 * Epoch (valid_es): 100% 157/157 [00:14<00:00, 11.07it/s, loss=1.303]
10/30 * Epoch (valid_it): 100% 157/157 [00:14<00:00, 11.07it/s, loss=0.030]
10/30 * Epoch (valid_tr): 100% 188/188 [00:16<00:00, 11.07it/s, loss=0.023]
[2020-04-16 11:52:26,700] 
10/30 * Epoch 10 (_base): lr=2.386e-07 | momentum=0.9000
10/30 * Epoch 10 (train): auc/_mean=0.9727 | auc/class_0=0.9727 | loss=0.1153
10/30 * Epoch 10 (valid): auc/_mean=0.8067 | auc/class_0=0.8067 | es_auc/_mean=0.8196 | es_auc/class_0=0.8196 | es_loss=0.5284 | it_auc/_mean=0.7930 | it_auc/class_0=0.7930 | it_loss=0.4937 | loss=0.4467 | tr_auc/_mean=0.8236 | tr_auc/class_0=0.8236 | tr_loss=0.3398
10/30 * Epoch 10 (valid_es): auc/_mean=0.8196 | auc/class_0=0.8196 | loss=0.5284
10/30 * Epoch 10 (valid_it): auc/_mean=0.7930 | auc/class_0=0.7930 | loss=0.4937
10/30 * Epoch 10 (valid_tr): 

18/30 * Epoch (train): 100% 13972/13972 [1:05:25<00:00,  3.56it/s, loss=0.007]
18/30 * Epoch (valid): 100% 500/500 [00:44<00:00, 11.22it/s, loss=0.826]
18/30 * Epoch (valid_es): 100% 157/157 [00:14<00:00, 11.13it/s, loss=1.668]
18/30 * Epoch (valid_it): 100% 157/157 [00:14<00:00, 11.14it/s, loss=0.008]
18/30 * Epoch (valid_tr): 100% 188/188 [00:16<00:00, 11.12it/s, loss=0.004]
[2020-04-16 20:50:41,021] 
18/30 * Epoch 18 (_base): lr=4.294e-07 | momentum=0.9000
18/30 * Epoch 18 (train): auc/_mean=0.9846 | auc/class_0=0.9846 | loss=0.0891
18/30 * Epoch 18 (valid): auc/_mean=0.8209 | auc/class_0=0.8209 | es_auc/_mean=0.8338 | es_auc/class_0=0.8338 | es_loss=0.6508 | it_auc/_mean=0.7960 | it_auc/class_0=0.7960 | it_loss=0.6344 | loss=0.5821 | tr_auc/_mean=0.8354 | tr_auc/class_0=0.8354 | tr_loss=0.4815
18/30 * Epoch 18 (valid_es): auc/_mean=0.8338 | auc/class_0=0.8338 | loss=0.6508
18/30 * Epoch 18 (valid_it): auc/_mean=0.7960 | auc/class_0=0.7960 | loss=0.6344
18/30 * Epoch 18 (valid_tr): 

26/30 * Epoch (train): 100% 13972/13972 [1:05:23<00:00,  3.56it/s, loss=9.339e-04]
26/30 * Epoch (valid): 100% 500/500 [00:44<00:00, 11.17it/s, loss=0.916]    
26/30 * Epoch (valid_es): 100% 157/157 [00:14<00:00, 11.07it/s, loss=1.858]
26/30 * Epoch (valid_it): 100% 157/157 [00:14<00:00, 11.07it/s, loss=0.002]
26/30 * Epoch (valid_tr): 100% 188/188 [00:16<00:00, 11.09it/s, loss=0.001]   
[2020-04-17 05:48:43,423] 
26/30 * Epoch 26 (_base): lr=6.203e-07 | momentum=0.9000
26/30 * Epoch 26 (train): auc/_mean=0.9913 | auc/class_0=0.9913 | loss=0.0679
26/30 * Epoch 26 (valid): auc/_mean=0.8038 | auc/class_0=0.8038 | es_auc/_mean=0.8235 | es_auc/class_0=0.8235 | es_loss=0.8007 | it_auc/_mean=0.7832 | it_auc/class_0=0.7832 | it_loss=0.8565 | loss=0.7497 | tr_auc/_mean=0.7968 | tr_auc/class_0=0.7968 | tr_loss=0.6174
26/30 * Epoch 26 (valid_es): auc/_mean=0.8235 | auc/class_0=0.8235 | loss=0.8007
26/30 * Epoch 26 (valid_it): auc/_mean=0.7832 | auc/class_0=0.7832 | loss=0.8565
26/30 * Epoch 26 (